In [1]:
from qiskit_metal import designs
from qiskit_metal import MetalGUI, Dict

design = designs.DesignPlanar()
gui = MetalGUI(design)

design.variables['cpw_width'] = '10 um'
design.variables['cpw_gap'] = '6 um'

In [ ]:
# -*- coding: utf-8 -*-

# This code is part of Qiskit.
#
# (C) Copyright IBM 2017, 2021.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.
"""File contains dictionary for NSquareSpiral and the make()."""

from qiskit_metal import draw, Dict
from qiskit_metal.qlibrary.core import QComponent
import numpy as np


class NSquareSpiral(QComponent):
    """A n count square spiral.

    Inherits `QComponent` class.

    Description:
        A n count square spiral.
        ::

            ____________
            |   _____   |
            |  |     |  |
            |  |     |  |
            |  |________|
            |

    .. image::
        NSquareSpiral.png

    .. meta::
        N Square Spiral

    Default Options:
        Convention: Values (unless noted) are strings with units included,
        (e.g., '30um')

        * n: '3' -- Number of turns of the spiral
        * width: '1um' -- the width of the line of the spiral
        * radius: '40um' -- The 'radius' of the inner portion of the spiral
        * gap: '4um' -- The distance between each layer of the spiral
        * subtract: 'False'
        * helper: 'False'
    """

    default_options = Dict(n='3',
                           width='1um',
                           radius='40um',
                           gap='4um',
                           subtract='False',
                           helper='False')
    """Default drawing options"""

    TOOLTIP = """An n count square spiral"""

    def make(self):
        """The make function implements the logic that creates the geoemtry
        (poly, path, etc.) from the qcomponent.options dictionary of
        parameters, and the adds them to the design, using
        qcomponent.add_qgeometry(...), adding in extra needed information, such
        as layer, subtract, etc."""
        p = self.p  # p for parsed parameters. Access to the parsed options.
        n = int(p.n)
        # Create the geometry

        spiral_list = []

        for step in range(n):
            point_value = p.radius / 2 + step * (p.width + p.gap)
            spiral_list.append((-point_value, -point_value))
            spiral_list.append((point_value, -point_value))
            spiral_list.append((point_value, point_value))
            spiral_list.append((-point_value - (p.width + p.gap), point_value))

        point_value = p.radius / 2 + (step + 1) * (p.width + p.gap)
        spiral_list.append((-point_value, -point_value))
        spiral_list = draw.LineString(spiral_list)

        spiral_list = draw.rotate(spiral_list, p.orientation, origin=(0, 0))
        spiral_list = draw.translate(spiral_list, p.pos_x, p.pos_y)

        ##############################################
        # add qgeometry
        self.add_qgeometry('path', {'n_spiral': spiral_list},
                           width=p.width,
                           subtract=p.subtract,
                           helper=p.helper,
                           layer=p.layer,
                           chip=p.chip)

        points = np.array(spiral_list.coords)
        # FIX POINTS,
        self.add_pin('spiralPin',
                     points=points[-2:],
                     width=p.width,
                     input_as_norm=True)


In [1]:
import numpy as np
from collections import OrderedDict

def pins_for_spiral(radius,gap,n):
    spiral_list = OrderedDict()
    for step in range(n):
        point_value = radius / 2 + step * (0 + gap)
        spiral_list[step*4] = np.array([-point_value, -point_value])
        spiral_list[step*4+1] = np.array([point_value, -point_value])
        spiral_list[step*4+2] = np.array([point_value, point_value])
        spiral_list[step*4+3] = np.array([-point_value - (0 + gap), point_value])

    point_value = radius / 2 + (step + 1) * (0 + gap)
    spiral_list[step*4+4] = np.array([-point_value, -point_value])
    return(spiral_list)

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from qiskit_metal import designs
from qiskit_metal import MetalGUI, Dict

design = designs.DesignPlanar()
gui = MetalGUI(design)

# if you disable the next line, then you will need to delete a component [<component>.delete()] before recreating it
design.overwrite_enabled = True

In [4]:
# for pins:
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
# for routing:
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.tlines.framed_path import RouteFramed

from qiskit_metal import draw, Dict
# for jogs:
from collections import OrderedDict

In [50]:
pin_opt = Dict( hfss_wire_bonds = True,pin_inputs=Dict(start_pin=Dict(
                                    component='open1i',
                                    pin='open'),
                                end_pin=Dict(
                                    component='short1o',
                                    pin='short')))

In [69]:
design.delete_all_components()  #needed only for rebuilds. will get a warning

otg11 = OpenToGround(design, 'open1i', options=dict(pos_x='0mm',  pos_y='0mm', orientation='0'))
otg12 = ShortToGround(design, 'short1o', options=dict(pos_x='0.2mm',  pos_y='1mm', orientation='90'))
# otg3 = ShortToGround(design,x)

gui.rebuild()
gui.autoscale()

In [77]:
anchors = pins_for_spiral(0.2,0.05,3)
ops=dict(fillet='40um')

In [80]:
anchors

OrderedDict([(0, array([-0.1, -0.1])),
             (1, array([ 0.1, -0.1])),
             (2, array([0.1, 0.1])),
             (3, array([-0.15,  0.1 ])),
             (4, array([-0.15, -0.15])),
             (5, array([ 0.15, -0.15])),
             (6, array([0.15, 0.15])),
             (7, array([-0.2 ,  0.15])),
             (8, array([-0.2, -0.2])),
             (9, array([ 0.2, -0.2])),
             (10, array([0.2, 0.2])),
             (11, array([-0.25,  0.2 ])),
             (12, array([-0.25, -0.25]))])

In [87]:
options = {'pin_inputs': 
            {'start_pin': {'component': 'open1i', 'pin': 'open'}, 
             'end_pin': {'component': 'short1o', 'pin': 'short'}},
            'lead': {'start_straight': '0um', 'end_straight': '0um'},
            'step_size': '0.25mm',
            'anchors': anchors,
            'total_length':'20mm',
            **ops
           }

In [88]:
design.delete_component('cpw_1')
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
qa = RoutePathfinder(design, 'cpw_1', options)

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)



In [89]:
gui.rebuild()
gui.autoscale()

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)

